## Getting and saving the cleaned twitter data
* We get the necessary columns for the twitter data from the  .json files by running the code on the cluster and saving the dataframes as parquet.
* The code to get the data is in cleaned_twitter.py

In [1]:
import pandas as pd
import os
from tqdm import tqdm
from pyspark import SparkConf, SparkContext
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("ADA-gcl")
#sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
#sqlContext.setConf("spark.sql.parquet.compression.codec", "snappy")


## Load the paqueret data

In [2]:
base_path = 'file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/Data/'
files = os.listdir('Data')

In [3]:
path_jan = 'file:///home/kirtan/Academics/EPFL/sem1/ADA/ADA-Project/Data/twitter_cleaned_jan.parquet'
df = sqlContext.read.parquet(path_jan)

In [4]:
df_twitter = df

In [5]:
for f in tqdm(files[1:]):
    temp = sqlContext.read.parquet(base_path + f)
    df_twitter = df_twitter.unionAll(temp)
   
    

In [ ]:
# Runs out of memory for me, but you can try. Function to convert df to pandas
#final = df_twitter.toPandas()

In [7]:
# Number of tweets
df_twitter.count()

10534806

In [12]:
# Removing those which do not have a sentiment
t1 = df_twitter.filter(df.sentiment != 'null')


In [13]:
t1.count()

9797616

In [21]:
from pyspark.sql.functions import col
# See main languages
t1.groupBy(['language']).count().sort(col("count").desc()).collect()

[Row(language='en', count=4129564),
 Row(language='de', count=2288993),
 Row(language='fr', count=2090395),
 Row(language='es', count=735317),
 Row(language='it', count=242370),
 Row(language='pt', count=60368),
 Row(language='ja', count=28238),
 Row(language='sv', count=24039),
 Row(language='ar', count=22622),
 Row(language='in', count=22321),
 Row(language='tr', count=20801),
 Row(language='da', count=16726),
 Row(language='tl', count=15504),
 Row(language='ht', count=15075),
 Row(language='pl', count=13937),
 Row(language='nl', count=12099),
 Row(language='no', count=10216),
 Row(language='ru', count=9812),
 Row(language='fi', count=8950),
 Row(language='et', count=8065),
 Row(language='hi', count=4597),
 Row(language='ko', count=4335),
 Row(language='el', count=4003),
 Row(language='lt', count=1776),
 Row(language='hu', count=1165),
 Row(language='sl', count=926),
 Row(language='lv', count=913),
 Row(language='uk', count=765),
 Row(language='zh', count=747),
 Row(language='is', co

In [27]:
# Keeping only the major languages
t2 = t1.where(col("language").isin({"en", "de", "fr"}))

In [28]:
t2.count()

8508952